In [1]:
import hopsworks
import pandas as pd
from prophet import Prophet
from datetime import datetime, timedelta, timezone
import joblib
import matplotlib.pyplot as plt
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.dates as mdates
import plotly.graph_objects as go
import parking

# === Step 1: Connect to Hopsworks ===
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

prev_hours=5
future_hours=5

recent_df = parking.get_parking_last_hour(hours=prev_hours)
recent_df

2025-10-23 03:40:35,350 INFO: generated new fontManager
2025-10-23 03:40:36,097 INFO: Python Engine initialized.

Logged in to project, explore it here https://hopsworks-stackit.hops.works/p/11384


,carpark_id,carpark_name,timestamp,value,vendor
0,mitte,Parkhaus Mitte,1761180000004,3,MSR
1,mitte,Parkhaus Mitte,1761180120008,3,MSR
2,mitte,Parkhaus Mitte,1761180180004,3,MSR
3,mitte,Parkhaus Mitte,1761180240006,3,MSR
4,mitte,Parkhaus Mitte,1761180300018,3,MSR
...,...,...,...,...,...
174,mitte,Parkhaus Mitte,1761190500004,5,MSR
175,mitte,Parkhaus Mitte,1761190560005,5,MSR
176,mitte,Parkhaus Mitte,1761190620002,6,MSR
177,mitte,Parkhaus Mitte,1761190680004,6,MSR


In [2]:
# model_name = "prophet_parking_forecast"

# # Get the latest version
# model_mr = mr.get_model(model_name, version=10)
# model_dir = model_mr.download()
# print(f"✅ Model downloaded to {model_dir}")

# model_path = f"{model_dir}/{model_name}.pkl"
# model = joblib.load(model_path)

✅ Model downloaded to /tmp/b2627e77-75fa-4b8e-9689-6bc3a0a92ce5/prophet_parking_forecast/10


In [3]:
recent_df["ds"] = pd.to_datetime(recent_df["timestamp"], unit="ms")  # <- FIXED
recent_df["y"] = recent_df["value"].astype(float)

print(f"✅ Loaded {len(recent_df)} records")
recent_df

✅ Loaded 179 records


,carpark_id,carpark_name,timestamp,value,vendor,ds,y
0,mitte,Parkhaus Mitte,1761180000004,3,MSR,2025-10-23 00:40:00.004,3.0
1,mitte,Parkhaus Mitte,1761180120008,3,MSR,2025-10-23 00:42:00.008,3.0
2,mitte,Parkhaus Mitte,1761180180004,3,MSR,2025-10-23 00:43:00.004,3.0
3,mitte,Parkhaus Mitte,1761180240006,3,MSR,2025-10-23 00:44:00.006,3.0
4,mitte,Parkhaus Mitte,1761180300018,3,MSR,2025-10-23 00:45:00.018,3.0
...,...,...,...,...,...,...,...
174,mitte,Parkhaus Mitte,1761190500004,5,MSR,2025-10-23 03:35:00.004,5.0
175,mitte,Parkhaus Mitte,1761190560005,5,MSR,2025-10-23 03:36:00.005,5.0
176,mitte,Parkhaus Mitte,1761190620002,6,MSR,2025-10-23 03:37:00.002,6.0
177,mitte,Parkhaus Mitte,1761190680004,6,MSR,2025-10-23 03:38:00.004,6.0


In [4]:
fg = fs.get_feature_group("parking")
df = fg.read()

# === Step 3: Prepare data for Prophet ===
df = df.sort_values("timestamp")
df = df.tail(5000)

# Prophet requires timezone-naive datetimes (no tzinfo)
df["ds"] = pd.to_datetime(df["timestamp"], unit="ms")  # <- FIXED
df["y"] = df["value"].astype(float)

# Filter for Parkhaus Mitte only
df = df[df["carpark_id"] == "mitte"]

# df = df.merge(recent_df)
df

2025-10-23 03:40:45,238 WARNING: VersionWarning: No version provided for getting feature group `parking`, defaulting to `1`.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.84s) 


,carpark_id,carpark_name,vendor,timestamp,value,ds,y
255035,mitte,Parkhaus Mitte,MSR,1760355600009,274,2025-10-13 11:40:00.009,274.0
246971,mitte,Parkhaus Mitte,MSR,1760355660007,274,2025-10-13 11:41:00.007,274.0
22851,mitte,Parkhaus Mitte,MSR,1760355720006,274,2025-10-13 11:42:00.006,274.0
297094,mitte,Parkhaus Mitte,MSR,1760355780010,275,2025-10-13 11:43:00.010,275.0
96962,mitte,Parkhaus Mitte,MSR,1760355840004,275,2025-10-13 11:44:00.004,275.0
...,...,...,...,...,...,...,...
245943,mitte,Parkhaus Mitte,MSR,1761155700006,89,2025-10-22 17:55:00.006,89.0
245973,mitte,Parkhaus Mitte,MSR,1761155760005,88,2025-10-22 17:56:00.005,88.0
245939,mitte,Parkhaus Mitte,MSR,1761155820007,88,2025-10-22 17:57:00.007,88.0
245969,mitte,Parkhaus Mitte,MSR,1761155880009,88,2025-10-22 17:58:00.009,88.0


In [5]:
# === Step 6: Forecast the next 3 hours (future only) ===
future = model.make_future_dataframe(periods=future_hours*60, freq="min")
forecast = model.predict(future)

# Keep only the *future* horizon (after the last known timestamp)
# last_ts = df["ds"].max()
# forecast_future = forecast[forecast["ds"] > last_ts].copy()
# last_ts_f = forecast["ds"].max()

# print(f"{last_ts_f}")
# print(f"{last_ts_f}")

forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-10-14 15:02:00.014,139.351950,110.561334,256.263213,139.351950,139.351950,45.534208,45.534208,45.534208,24.694291,24.694291,24.694291,20.839916,20.839916,20.839916,0.0,0.0,0.0,184.886157
1,2024-10-14 16:38:00.009,139.190088,72.316079,217.714827,139.190088,139.190088,3.544653,3.544653,3.544653,-21.190568,-21.190568,-21.190568,24.735221,24.735221,24.735221,0.0,0.0,0.0,142.734740
2,2024-10-14 17:36:00.007,139.092296,58.133851,208.793565,139.092296,139.092296,-8.255492,-8.255492,-8.255492,-35.141867,-35.141867,-35.141867,26.886374,26.886374,26.886374,0.0,0.0,0.0,130.836804
3,2024-10-14 19:30:00.005,138.900085,46.736891,195.743584,138.900085,138.900085,-17.027320,-17.027320,-17.027320,-47.684273,-47.684273,-47.684273,30.656952,30.656952,30.656952,0.0,0.0,0.0,121.872765
4,2024-10-14 21:34:00.004,138.691014,31.028528,174.122375,138.691014,138.691014,-35.350896,-35.350896,-35.350896,-69.413627,-69.413627,-69.413627,34.062731,34.062731,34.062731,0.0,0.0,0.0,103.340118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5295,2025-10-17 02:37:00.003,98.440938,-39.007688,103.382546,98.440938,98.440938,-68.883380,-68.883380,-68.883380,-73.714406,-73.714406,-73.714406,4.831026,4.831026,4.831026,0.0,0.0,0.0,29.557558
5296,2025-10-17 02:38:00.003,98.441544,-44.170620,106.519627,98.441544,98.441544,-69.049735,-69.049735,-69.049735,-73.868292,-73.868292,-73.868292,4.818557,4.818557,4.818557,0.0,0.0,0.0,29.391809
5297,2025-10-17 02:39:00.003,98.442149,-45.462542,104.471670,98.442149,98.442149,-69.214759,-69.214759,-69.214759,-74.020851,-74.020851,-74.020851,4.806092,4.806092,4.806092,0.0,0.0,0.0,29.227390
5298,2025-10-17 02:40:00.003,98.442755,-47.475587,96.647108,98.442755,98.442755,-69.378374,-69.378374,-69.378374,-74.172005,-74.172005,-74.172005,4.793631,4.793631,4.793631,0.0,0.0,0.0,29.064381


In [ ]:
# === Step 7: Use Prophet's forecast DataFrame directly ===
merged_forecast = forecast_future.copy()

# Add metadata columns
basetime = int(datetime.now(timezone.utc).timestamp() * 1000)  # bigint UTC ms
merged_forecast["basetime"] = basetime
merged_forecast["carpark_id"] = "mitte"
merged_forecast["carpark_name"] = "Parkhaus Mitte"
merged_forecast["vendor"] = "MSR"

# Keep relevant columns
merged_forecast = merged_forecast[[
    "carpark_id", "carpark_name", "vendor", "ds",
    "yhat", "yhat_lower", "yhat_upper", "basetime"
]]

# Rename ds → timestamp
merged_forecast = merged_forecast.rename(columns={"ds": "timestamp"})

# Convert timestamp to bigint (ms)
merged_forecast["timestamp"] = merged_forecast["timestamp"].astype("int64") // 10**6
merged_forecast["basetime"] = merged_forecast["basetime"].astype("int64")

print(f"✅ Created forecast DataFrame with {len(merged_forecast)} rows")

# === Step 7: Plot 3-hour forecast only ===
fig = go.Figure()

# Predicted trend
fig.add_trace(go.Scatter(
    x=forecast_future["ds"], y=forecast_future["yhat"],
    mode="lines", name="Forecast (next 3 hrs)",
    line=dict(color="blue", width=2)
))

# Confidence interval
fig.add_trace(go.Scatter(
    x=pd.concat([forecast_future["ds"], forecast_future["ds"][::-1]]),
    y=pd.concat([forecast_future["yhat_upper"], forecast_future["yhat_lower"][::-1]]),
    fill='toself', fillcolor='rgba(0, 0, 255, 0.1)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip", showlegend=True, name="Confidence Interval"
))

fig.update_layout(
    title="Prophet Forecast — Next 3 Hours (Parkhaus Mitte)",
    xaxis_title="Time",
    yaxis_title="Predicted Free Parking Spaces",
    hovermode="x unified",
    template="plotly_white",
    height=500
)

fig.show()



In [ ]:
# === Step 10: Write merged forecast to a new feature group ===
forecast_fg = fs.get_or_create_feature_group(
    name="forecasts",
    version=1,
    primary_key=["basetime"],
    description="Forecasts of free parking spaces for Parkhaus Mitte using Prophet",
    online_enabled=False
)

forecast_fg.insert(merged_forecast)